# S3W8D5: 记忆机制与上下文 (Memory Management)

今天我们要给你的 AI 注入“灵魂”。没有记忆的 AI 就像一条金鱼（传说只有 7 秒记忆），哪怕你刚才告诉过它你的名字，下一秒它就会问：“你好，请问你是谁？”

在算法岗面试中，**Memory (记忆)** 是连接 LLM 和 Agent 的桥梁，也是考察你对 **Context Window (上下文窗口)** 理解深度的最佳考点。

**🎯 今日目标**

1. **原理理解**：为什么 API 是无状态的？我们如何在客户端维护状态？
2. **核心实现**：手动编写 `ConversationBuffer` 和 `WindowBuffer`。
3. **工程难点**：实现 Token 计数与自动截断（Sliding Window），防止撑爆模型。


## 📖 1 理论知识讲解 (Theory & Analogy)

**为什么需要记忆模块？**

> **🧬 医学图像类比：病历 (Patient History)**
> * **Stateless (无状态)**：就像放射科医生只看**当前**这张 X 光片。他能看出“这里骨折了”，但他不知道“这已经是病人这周第三次骨折了”（缺乏上下文），可能会漏掉“骨质疏松”的诊断。
> * **Full Memory (全量记忆)**：把病人从出生到现在的**所有**病历都扔给医生。
> * *优点*：信息全。
> * *缺点*：医生看不完（Context Limit 限制），而且很贵（Token 计费）。
> 
> 
> * **Window Memory (滑动窗口)**：只给医生看**最近 3 次**的就诊记录。
> * **Summary Memory (摘要记忆)**：给医生看一份**出院小结**（比如：“患者有严重骨质疏松史”），而不是原始的流水账。
> 
> 

在 LLM 开发中，我们就需要根据场景选择不同的“病历管理策略”。

## 💻 2 代码实现

### 2.1 简单的Buffer Memory

In [2]:
from typing import List, Dict

class SimpleMemory:
    def __init__(self):
        self.messages = [] # 存储 {"role": "user", "content": "..."}

    def add_user_message(self, text: str):
        self.messages.append({"role": "user", "content": text})

    def add_ai_message(self, text: str):
        self.messages.append({"role": "assistant", "content": text})

    def get_history(self) -> List[Dict]:
        return self.messages

    def clear(self):
        self.messages = []

# 测试
memory = SimpleMemory()
memory.add_user_message("我叫黄同学")
memory.add_ai_message("你好，黄同学")
memory.add_user_message("我刚才说了我叫什么？")

print(memory.get_history())


[{'role': 'user', 'content': '我叫黄同学'}, {'role': 'assistant', 'content': '你好，黄同学'}, {'role': 'user', 'content': '我刚才说了我叫什么？'}]


### 2.2 结合 LLM 实战

让我们看看把这个 list 发给 LLM 会发生什么。

In [3]:
import sys
import os
sys.path.append(os.path.abspath("../../"))
from src.llm.client import LLMClient

client = LLMClient()
memory = SimpleMemory()

# 第一轮对话
user_input = "我叫黄同学，我是一名算法工程师。"
memory.add_user_message(user_input)

# 构造 Prompt：System + History
# 注意：这里我们需要手动把 list 拼成 prompt，或者直接发 list 给 client (如果 client 支持)
# 为了演示，假设我们的 client.generate 只接受 string，我们需要把 history 变成 text
def build_prompt(history, query):
    prompt_str = ""
    for msg in history:
        prompt_str += f"{msg['role']}: {msg['content']}\n"
    prompt_str += f"user: {query}\nassistant:"
    return prompt_str

# 这里我们稍微 hack 一下，直接把 history 塞进 client 的 messages 里会更标准
# 但为了演示原理，我们先看效果。
print("--- Round 1 ---")
# 这里的 client.generate 是 Week 8 Day 1 写的，它内部只接受 prompt string
# 如果要支持多轮，我们需要修改 client.generate 接受 messages 列表
# 或者我们简单地把 history 拼接到 prompt 里 (Stateless 的本质)

response = client.generate(f"History:\n{memory.get_history()}\n\nUser: 刚才我说我是什么职业？")
print(response)
memory.add_ai_message(response)

--- Round 1 ---
您刚才说您是一名**算法工程师**。


> **🛑 停顿思考**：随着对话进行，`memory.get_history()` 返回的列表会越来越长。如果聊了 1000 轮，API 就会报错：`ContextLengthExceeded`。

### 2.3 滑动窗口与 Token 截断 (Window Memory) 🚀

这是工程中最实用的部分。我们需要限制“记忆”的大小。

In [4]:
# Cell 3
import tiktoken

class WindowMemory:
    def __init__(self, max_tokens: int = 500):
        self.messages = []
        self.max_tokens = max_tokens
        # 加载编码器 (GPT-3.5/4 标准)
        self.encoder = tiktoken.get_encoding("cl100k_base")

    def count_tokens(self, text: str) -> int:
        return len(self.encoder.encode(text))

    def add_message(self, role: str, content: str):
        self.messages.append({"role": role, "content": content})
        self._trim_memory()

    def _trim_memory(self):
        """核心逻辑：如果超长，就从头开始扔，直到满足要求"""
        while True:
            # 计算当前总长度
            total_tokens = sum([self.count_tokens(m["content"]) for m in self.messages])
            if total_tokens <= self.max_tokens:
                break
            
            # 弹出最早的一条 (FIFO - 先进先出)
            # 注意：通常要成对弹出 (User+AI)，否则对话会错位，这里简单处理弹出一个
            if self.messages:
                removed = self.messages.pop(0)
                print(f"⚠️ Memory Full! Popping: {removed['content'][:10]}...")
            else:
                break

# 测试截断
window_mem = WindowMemory(max_tokens=20) # 故意设得很小
window_mem.add_message("user", "你好，我是黄同学。")
window_mem.add_message("assistant", "你好！")
window_mem.add_message("user", "今天天气真不错，适合出去玩。") # 这句加进去可能就爆了
window_mem.add_message("assistant", "是的，你想去哪里？")

print("\n--- Current Memory ---")
for msg in window_mem.messages:
    print(msg)

⚠️ Memory Full! Popping: 你好，我是黄同学。...
⚠️ Memory Full! Popping: 你好！...
⚠️ Memory Full! Popping: 今天天气真不错，适合...

--- Current Memory ---
{'role': 'assistant', 'content': '是的，你想去哪里？'}


## 🧠 3 深度理论 & 面试必问 (Deep Dive)

### 🔥 Q1: 如果对话历史太长超过 Context Window，有哪些处理策略？

这是 Agent 开发中的核心难题。

**标准回答 (由浅入深)：**

1. **直接截断 (Truncation / Sliding Window)**：
* **原理**：保留最近的  轮对话 (FIFO 队列)。
* **优点**：实现简单，保留了最新的细节。
* **缺点**：丢失了早期的关键信息（比如用户刚开始说的名字）。
* *LC 关联*：这不就是 **Queue** 吗？


2. **摘要总结 (Summarization)**：
* **原理**：每隔几轮，触发一个 LLM 调用，让它把之前的对话压缩成一段 Summary。
* *Prompt*："请把以上对话总结为 100 字以内的摘要，保留关键实体。"
* **缺点**：增加了 API 调用成本，且摘要会丢失细节。


3. **向量检索 (Vector Store / Long-term Memory)**：
* **原理**：把历史对话切片存入向量数据库 (Chroma/Faiss)。当用户问新问题时，先去数据库里 Retrieve 相关的历史记录。
* **场景**：用户问“我们上周聊到的那个法律条款是什么？”，这时候滑动窗口早忘了，但向量检索能找回来。这是 **RAG** 的思想在 Memory 上的应用。



### 🔥 Q2: 为什么不把所有历史都发给模型？

1. **钱 (Cost)**：API 是按 Token 收费的。每轮都发全文，费用是  增长的。
2. **延迟 (Latency)**：输入越长，Prefill (首字生成时间) 越慢。
3. **性能 (Performance)**：研究表明（Lost in the Middle 现象），当 Context 极长时，模型容易忽略中间的信息，只记得开头和结尾。

## 🏗️ 4 工程封装 (Engineering)

请在 `src/llm/memory.py` 中封装代码。我们将实现一个结合了 Buffer 和 Window 的类。


```python
# src/llm/memory.py
from typing import List, Dict, Optional
import tiktoken

class MemoryBuffer:
    def __init__(self, max_tokens: Optional[int] = 2000, system_prompt: str = ""):
        self.messages = []
        if system_prompt:
            self.messages.append({"role": "system", "content": system_prompt})
        
        self.max_tokens = max_tokens
        self.encoder = tiktoken.get_encoding("cl100k_base")

    def add(self, role: str, content: str):
        self.messages.append({"role": role, "content": content})
        if self.max_tokens:
            self._trim()

    def get_context(self) -> List[Dict]:
        return self.messages

    def _trim(self):
        # 简单粗暴的修剪策略：保留 System Prompt，修剪中间的
        while self._count_total_tokens() > self.max_tokens:
            # 确保不删掉 system prompt (index 0)
            if len(self.messages) > 1:
                # 删除 index 1 (最早的非 system 消息)
                self.messages.pop(1)
            else:
                break
                
    def _count_total_tokens(self) -> int:
        return sum([len(self.encoder.encode(m["content"])) for m in self.messages])

    def clear(self):
        # 保留 system prompt
        system = self.messages[0] if self.messages and self.messages[0]['role'] == 'system' else None
        self.messages = []
        if system:
            self.messages.append(system)

```

## 🧩 6. 今日 LeetCode 练习

记忆管理本质上就是 **Queue (队列)** 的操作。

* **题目**: **LC 933. Number of Recent Calls (最近的请求次数)** - 简单
* **关联**: 这道题要求你只统计过去 3000ms 内的请求。
* 这完全就是 **Window Memory** 的逻辑！
* 一旦时间戳超出了窗口（`t - 3000`），就 `popleft`。这和我们刚才写的 `_trim()` 逻辑一模一样。

## ✅ 7. 今日任务总结

* [ ] 安装 `tiktoken`。
* [ ] 理解了 Memory 就是 List，Context Window 是物理限制。
* [ ] 实现了 Token 计数和 Sliding Window 截断逻辑。
* [ ] 封装了 `src/llm/memory.py`。
* [ ] **面试准备**：可以流利说出处理长文本的三种策略（截断、摘要、检索）。